In [1]:
import csv
import os
from collections import defaultdict
import pandas as pd
from nltk.corpus import stopwords
from textblob import TextBlob, Word
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors # load the Stanford GloVe model
#nltk.download()

C:\Users\Naini\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
os.getcwd()
os.chdir("C:\\Users\\Naini\\final-project\\News-Headline-Generation\\data")

### 1. Basic Feature Extraction

In [3]:
#reading csv
train = pd.read_csv('output.csv', encoding='ISO-8859-1',low_memory=False)

In [4]:
train = train['content']
train

0         WASHINGTON  Ã¢ÂÂ   Congressional Republicans...
1         After the bullet shells get counted, the blood...
2         When Walt DisneyÃ¢ÂÂs Ã¢ÂÂBambiÃ¢ÂÂ open...
3         Death may be the great equalizer, but it isnÃ¢...
4         SEOUL, South Korea  Ã¢ÂÂ   North KoreaÃ¢ÂÂ...
5         LONDON  Ã¢ÂÂ   Queen Elizabeth II, who has b...
6         BEIJING  Ã¢ÂÂ   President Tsai   of Taiwan s...
7         Danny Cahill stood, slightly dazed, in a blizz...
8         Just how   is Hillary Kerr, the    founder of ...
9         Angels are everywhere in the MuÃÂ±iz familyÃ¢...
10        With Donald J. Trump about to take control of ...
11        THOMPSONS, Tex.  Ã¢ÂÂ   Can one of the most ...
12        WEST PALM BEACH, Fla.  Ã¢ÂÂ   When   Donald ...
13        This article is part of a series aimed at help...
14        ItÃ¢ÂÂs the season for family travel and pho...
15        Finally. The Second Avenue subway opened in Ne...
16          pages into the   journal fou

In [6]:
train = train[train.notnull()]
train

0         WASHINGTON  Ã¢ÂÂ   Congressional Republicans...
1         After the bullet shells get counted, the blood...
2         When Walt DisneyÃ¢ÂÂs Ã¢ÂÂBambiÃ¢ÂÂ open...
3         Death may be the great equalizer, but it isnÃ¢...
4         SEOUL, South Korea  Ã¢ÂÂ   North KoreaÃ¢ÂÂ...
5         LONDON  Ã¢ÂÂ   Queen Elizabeth II, who has b...
6         BEIJING  Ã¢ÂÂ   President Tsai   of Taiwan s...
7         Danny Cahill stood, slightly dazed, in a blizz...
8         Just how   is Hillary Kerr, the    founder of ...
9         Angels are everywhere in the MuÃÂ±iz familyÃ¢...
10        With Donald J. Trump about to take control of ...
11        THOMPSONS, Tex.  Ã¢ÂÂ   Can one of the most ...
12        WEST PALM BEACH, Fla.  Ã¢ÂÂ   When   Donald ...
13        This article is part of a series aimed at help...
14        ItÃ¢ÂÂs the season for family travel and pho...
15        Finally. The Second Avenue subway opened in Ne...
16          pages into the   journal fou

In [ ]:
#Number of stopwords
#stop = stopwords.words('english')

In [ ]:
#train['stopwords'] = train['content'].apply(lambda x: len([x for x in x.split() if x in stop]))
#train[['content','stopwords']].head()

### 2. Basic Pre-processing

In [ ]:
#transform data into lower case
train = train.apply(lambda x: " ".join(x.lower() for x in x.split()))
train.head()

In [7]:
#Removing Punctuation
train = train.str.replace('[^\w\s]','')
train

0         WASHINGTON  ÃÂÂ   Congressional Republicans ha...
1         After the bullet shells get counted the blood ...
2         When Walt DisneyÃÂÂs ÃÂÂBambiÃÂÂ opened in 194...
3         Death may be the great equalizer but it isnÃÂÂ...
4         SEOUL South Korea  ÃÂÂ   North KoreaÃÂÂs leade...
5         LONDON  ÃÂÂ   Queen Elizabeth II who has been ...
6         BEIJING  ÃÂÂ   President Tsai   of Taiwan shar...
7         Danny Cahill stood slightly dazed in a blizzar...
8         Just how   is Hillary Kerr the    founder of a...
9         Angels are everywhere in the MuÃÂiz familyÃÂÂs...
10        With Donald J Trump about to take control of t...
11        THOMPSONS Tex  ÃÂÂ   Can one of the most promi...
12        WEST PALM BEACH Fla  ÃÂÂ   When   Donald J Tru...
13        This article is part of a series aimed at help...
14        ItÃÂÂs the season for family travel and photos...
15        Finally The Second Avenue subway opened in New...
16          pages into the   journal fou

In [8]:
train.to_csv('output_cleaned_body.csv', index=False)

In [ ]:
#Removal of Stop Words
#train['content'] = train['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#train['content'].head()

In [ ]:
#Common word removal
#freq = pd.Series(' '.join(train['content']).split()).value_counts()[:10]
#freq

In [ ]:
#freq = list(freq.index)
#train['content'] = train['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#train['content'].head()

In [ ]:
#Rare words removal
#rare = pd.Series(' '.join(train['content']).split()).value_counts()[-10:]
#rare

In [ ]:
#rare = list(rare.index)
#train['content'] = train['content'].apply(lambda x: " ".join(x for x in x.split() if x not in rare))
#train['content'].head()

In [ ]:
#Tokenization - dividing the text into a sequence of words or sentences
#we have used the textblob library to first transform our data into a blob and then converted them into a series of words
tokenized_words=[]
#for i,x in enumerate(train['content']):
    #if(len(x) > 1 ):
       #tokenized_words = TextBlob(x).words

In [ ]:
#Stemming -  removal of suffices, like “ing”, “ly”, “s”
#st = PorterStemmer()
#train['content'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [ ]:
#Lemmatization - it converts the word into its root word
#train['content'] = train['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
#train['content'].head()

### 3. Advance Text Processing
 

In [ ]:
#N-grams - combination of multiple words used together.
#j=[]
#for i,x in enumerate(train['content']):
    #j = TextBlob(x).ngrams(2)




In [ ]:
# Term frequency - ratio of the count of a word present in a sentence, to the length of the sentence
#tf1 = (train['content']).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
#tf1.columns = ['words','tf']
#tf1

In [ ]:
#Inverse Document Frequency - log of the ratio of the total number of rows to the number of rows in which that word is present
#import numpy as np
#for i,word in enumerate(tf1['words']):
  #tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['content'].str.contains(word)])))

#tf1

In [ ]:
#Term Frequency – Inverse Document Frequency (TF-IDF) - multiplication of the TF and IDF 
#tf1['tfidf'] = tf1['tf'] * tf1['idf']
#tf1


In [ ]:
#tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
#stop_words= 'english',ngram_range=(1,1))
#train_vect = tfidf.fit_transform(train['content'])

#train_vect


In [ ]:
#Bag of Words - representation of text which describes the presence of words within the text data
#bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
#train_bow = bow.fit_transform(train['content'])
#train_bow


In [ ]:
# Word Embeddings
#glove_input_file = 'glove.6B.100d.txt'
#word2vec_output_file = 'glove.6B.100d.txt.word2vec'
# convert it into the word2vec format
#glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
#load the above word2vec file as a model
#filename = 'glove.6B.100d.txt.word2vec'
#model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [ ]:
# take the average to represent the string ‘go away’ in the form of vectors having 100 dimensions
#(model['music'] + model['family'])/2